In [0]:
# Transforma CNV em CSV

import pandas as pd
import re
import io

def convert_cnv_to_csv(cnv_content, output_file="/Volumes/workspace/sus_establishments/raw/sus_dictionary/natureza.csv"):
    lines = cnv_content.strip().split('\n')
    
    # 1. Ignorar a primeira linha de cabeçalho técnico
    data_lines = lines[1:]
    
    parsed_data = []
    
    for line in data_lines:
        if not line.strip():
            continue
            
        # A lógica do CNV: O código de mapeamento está sempre no FINAL da linha
        # A descrição está no meio.
        
        # Remove espaços extras nas extremidades
        clean_line = line.strip()
        
        # Regex para pegar:
        # 1. Um grupo de números/espaços inicial (ignoramos)
        # 2. A descrição (texto no meio)
        # 3. O código ou range (o que vem depois de vários espaços no final)
        match = re.search(r'^(?:\d+\s+)?(?:\d+\s+)?(.*?)\s{2,}([\d-]+)$', line)
        
        if match:
            descricao = match.group(1).strip()
            codigo = match.group(2).strip()
            parsed_data.append({"codigo": codigo, "descricao": descricao})
        else:
            # Fallback para linhas com formato simplificado (ex: a última linha)
            parts = clean_line.rsplit(None, 1)
            if len(parts) == 2:
                parsed_data.append({"codigo": parts[1], "descricao": parts[0]})

    # 2. Criar DataFrame
    df = pd.DataFrame(parsed_data)
    
    # 3. Salvar como CSV
    df.to_csv(output_file, index=False, encoding='utf-8', sep=';')
    print(f"✅ Sucesso! {len(df)} linhas processadas e salvas em: {output_file}")
    return df

with open('/Volumes/workspace/sus_establishments/raw/sus_dictionary/NATUREZA.CNV', 'r', encoding='ISO-8859-1') as f:
    cnv_text = f.read()

df_resultado = convert_cnv_to_csv(cnv_text)

# Visualizar as primeiras linhas
print(df_resultado.head())

In [0]:
# Transforma DBF em CSV
%pip install simpledbf

import contextlib
import os
import tempfile
import simpledbf

src = "/Volumes/workspace/sus_establishments/raw/sus_dictionary/CADGERSP.dbf"
dest = "/Volumes/workspace/sus_establishments/raw/sus_dictionary/CADGERSP.csv"
detected_encoding = 'MacRoman' 

simpledbf.Dbf5(src, codec=detected_encoding).to_csv(dest)
print(f"Conversion successful! '{src}' converted to '{dest}' using encoding '{detected_encoding}'.")

